# Когда нужна оценка подсветки?
1. При написании текстов (заголовков)  
2. При аудите кампаний


# Требования к продукту
 - Или он часть большого продукта по написанию текстов и сборки РК в целом.  
 - Или он делает аудит РК, поэтому сам стучится по API. И здесь меня останавливает только АПИ эдвордс. Ну и в Директе помню было много кода. Но АПИ - это неизбежность ;)
 
 
# Обьем MVP
 - загрузка экселек с колонками по стандарту
 - выгрузка стандартной эксельки с названием группы и скором


# Открытые вопросы
 - Нужно ли анализировать весь текст, или работать только с заголовком 1. В любом случае надо получить все заголовки, то есть работать с группами.


In [2]:
import pandas as pd
import pymorphy2

# TODO: парсить параметры из командной строки, определять тип файла и разметку автоматически
free_format_file = 'test_data/free_format_file.xlsx' # нужны библиотеки

# лематизатор
morph = pymorphy2.MorphAnalyzer()
# TODO: похоже нужен дополнительно свой словарик, напроимер "продать" и "продавать" - разные слова для pymorthy2 :(

# получаем исходные данные
data = pd.read_excel(free_format_file)



In [47]:
def normalize(sentense):
    norm = []
    for w in sentense.split():
        w = morph.parse(w)[0]
        w = w.normal_form
        norm.append(w)
    return norm

# TODO: отбрасывать пунктуацию
# TODO: отбрасывать предлоги по списку или длине слова

def evaluate(key_norm, head_norm):
#   для начала простая оценка - будем считать сколько слов из ключа есть в заголовке, в процентах
    score = 0
    for w in head_norm:
        if w in key_norm:
            score += 1
    return score/len(key_norm)


Столкнулся с тем, что PyMorthy2 не корректно отрабатывает слова "продал" и "продаем" - возвращает разные начальные формы.  

РАссматриваю вариант использовать яндексовский стеммер https://tech.yandex.ru/mystem/doc/index-docpage/. Надо будет разобраться как работать с внешней библиотекой. Вот пример вызова через файлы:  
    mystem.exe -lcs test.txt out.txt  
Красиво было бы использовать стандартный ввод/вывод.  

В выходном потоке стемера надо избавиться от {} и что-то сделать с несколькими вариантами разбора слова. Как вариант - оставить только первый вариант. Нужно читать доку.

In [59]:
data = data.assign(key_norm = data['Keyword'].map(normalize))
data = data.assign(head_norm = data['Headline 1'].map(normalize))
data = data.assign(score = data.apply(lambda row: evaluate(row.key_norm, row.head_norm), axis=1))
data

,Keyword,Headline 1,Another field,And one additional,key_norm,head_norm,score
0,ключевое слово,Заголовок с нулевой подсветкой,NaN,NaN,"[ключевой, слово]","[заголовок, с, нулевой, подсветка]",0.0
1,продажа слонов,Купите слона со скидкой!,NaN,NaN,"[продажа, слон]","[купить, слон, с, скидкой!]",0.5
2,продажа слонов,Продаем слона,NaN,NaN,"[продажа, слон]","[продавать, слон]",0.5
3,слон красный купить,Купите милого красного слона,NaN,NaN,"[слон, красный, купить]","[купить, милый, красный, слон]",1.0


# Исследуем особенности pymorthy2

In [61]:
w = morph.parse('продал')[0]
print(w.normal_form)

w = morph.parse('продаем')[0]
print(w.normal_form)


продать
продавать
